# **Models_BERT**

## Mount drive and import libraries

In [1]:
# Mount a drive
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Import libraries
import pickle
import numpy as np
import pandas as pd
import random
import collections
import os
# SK-learn libraries for learning.
# from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# NLP libraries
import nltk, re, pprint, string
# from nltk import word_tokenize
# nltk.download('punkt')

# Set seed
random.seed(8)
np.random.seed(seed=13)

## Import data

In [0]:
# Path to the file MoodyLyrics4Q_raw.pkl
path_lyrics = "/content/drive/My Drive/Project/2_clean_data/data_lyrics_clean.pkl"
# If David runs, use this path "/content/drive/My Drive/Fall 2019/W266/Project/2_clean_data/data_lyrics_clean.pkl"
# Unpickle the dataset in pandas dataframe
df = pd.read_pickle(path_lyrics)

## EDA

In [4]:
df.head(5)

,ml_index,mer_index,artist,title,mood,mood_pn,lyrics,lyrics_len
0,1.0,NaN,George Michael,I Want Your Sex,happy,pos,"There's things that you guess, And things that...",375
1,2.0,NaN,Rob Zombie,Pussy Liquor,angry,neg,"Baby: Gimme a B, Gimme a A, Gimme a B, Gimme a...",353
2,3.0,NaN,Katatonia,12,sad,neg,"Black theatre of love, Violet thatncers cast t...",88
3,4.0,NaN,Bing Crosby,Swinging On A Star,happy,pos,"Would you like to swing on a star, Carry moonb...",285
4,5.0,NaN,Ludacris,Get Back,angry,neg,"Hands up! Hands up, Here's another one, And a ...",835


In [5]:
# Explore the dataset
df.tail(5)

,ml_index,mer_index,artist,title,mood,mood_pn,lyrics,lyrics_len
2569,NaN,MT0033391277,Simon Garfunkel,April Come She Will,relaxed,pos,", April come she will, When streams are ripe a...",67
2570,NaN,MT0033733581,Sinead O'Connor,Door Peep,relaxed,pos,"Door peep shall not enter this a holy land, Wh...",171
2571,NaN,MT0033994043,Rosemary Clooney,Tenderly,relaxed,pos,The evening breeze caressed the trees tenderly...,70
2572,NaN,MT0034992204,John Denver,Farewell Andromeda (Welcome to My Morning),relaxed,pos,"Welcome to my morning, Welcome to my thaty, I'...",172
2573,NaN,MT0035119161,Elton John,Dixie Lily,relaxed,pos,"Showboat coming up the river, See her lanterns...",129


## Train/Test split

We will define a train/test split ratio

In [0]:
# test ratio portion over all observations
test_ratio = 0.2

### Pos/neg mood

In [0]:
# Split the data into random train and test subsets for pos/neg mood prediction
X_pn_train, X_pn_test, y_pn_train, y_pn_test = train_test_split(df['lyrics'], df['mood_pn'], test_size=test_ratio, random_state=8)

In [8]:
# Check train and test split
print(f'Train size: {len(y_pn_train)}')
print(f'Test size: {len(y_pn_test)}')


# Check balance in random subsets
print(f'\nPositive mood in train: {np.sum(y_pn_train=="pos")}')
print(f'Negative mood in train: {np.sum(y_pn_train=="neg")}')
print(f'\nPositive mood in test: {np.sum(y_pn_test=="pos")}')
print(f'Negative mood in test: {np.sum(y_pn_test=="neg")}')

Train size: 2059
Test size: 515

Positive mood in train: 1005
Negative mood in train: 1054

Positive mood in test: 250
Negative mood in test: 265


### 4Q mood

In [0]:
# Split the data into random train and test subsets for 4Q mood prediction
X_m_train, X_m_test, y_m_train, y_m_test = train_test_split(df['lyrics'], df['mood'], test_size=0.2, random_state=8)

In [10]:
# Check balance in random subsets: train
index_train = list(y_m_train.index)
df.loc[index_train, :].groupby(by='mood').count()

,ml_index,mer_index,artist,title,mood_pn,lyrics,lyrics_len
mood,,,,,,,
angry,370,159,529,529,529,529,529
happy,372,163,535,535,535,535,535
relaxed,354,116,470,470,470,470,470
sad,353,172,525,525,525,525,525


In [11]:
# Check balance in random subsets:test
index_test = list(y_m_test.index)
df.loc[index_test, :].groupby(by='mood').count()

,ml_index,mer_index,artist,title,mood_pn,lyrics,lyrics_len
mood,,,,,,,
angry,98,45,143,143,143,143,143
happy,84,48,132,132,132,132,132
relaxed,86,32,118,118,118,118,118
sad,90,32,122,122,122,122,122


## Models

The data is already splitted in train and test datasets.

### BERT

In [12]:
#BERT TF Library
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 2.2MB/s 


In [13]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

BERT Parameters

DATA_COLUMN is the column of information to be evaluated. In our case, our lyrics. LABEL_COLUMN is the column to be predicted. In our case, we will start with pos/neg and then the mood.


In [0]:
DATA_COLUMN = 'lyrics'
LABEL_COLUMN = 'mood_pn' #change to mood later
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = ['pos', 'neg'] # change to ['angry', 'sad', 'happy', 'relaxed']

BERT Pre-processing

*   `text_a` is the text we want to classify, which in this case, is the `lyrics` field in our Dataframe.
*   `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is text_b a translation of text_a? Is text_b an answer to the question asked by text_a?). This doesn't apply to our task, so we can leave `text_b` blank.
*   `label` is the `label` for our example, i.e. True, False

In [0]:
## Datasets
## Pos/Neg - X_pn_train, X_pn_test, y_pn_train, y_pn_test  
## Mood - X_m_train, X_m_test, y_m_train, y_m_test
train, test = train_test_split(df, test_size=0.2, random_state=8)

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

## Text Pre-processing

A large portion of the data was been cleaned up prior to this step, but BERT has specific formats that it is expecting for the model to work correctly.

In [18]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 2059


INFO:tensorflow:Writing example 0 of 2059


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: pos (id = 0)


INFO:tensorflow:label: pos (id = 0)


In [0]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabilities.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [0]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 16
LEARNING_RATE = 2e-6
NUM_TRAIN_EPOCHS = 5.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 5000
SAVE_SUMMARY_STEPS = 100

In [0]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [24]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpl_v1i_zn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbc41c67be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpl_v1i_zn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbc41c67be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [26]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 643 into /tmp/tmpl_v1i_zn/model.ckpt.


INFO:tensorflow:Saving checkpoints for 643 into /tmp/tmpl_v1i_zn/model.ckpt.


INFO:tensorflow:Loss for final step: 0.1563636.


INFO:tensorflow:Loss for final step: 0.1563636.


Training took time  0:10:49.883051


In [0]:
#!rm -rf /tmp/tmp92ds7hbn
#!rm -rf /tmp/tmpdrh8fwt6
#!rm -rf /tmp/tmpfola15fn
#!rm -rf /tmp/tmpg1cwiyr_
#!rm -rf /tmp/tmprb3dgg21
#!rm -rf /tmp/tmpynhurkdm

In [0]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [29]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 643: auc = 0.7826792, eval_accuracy = 0.7825243, f1_score = 0.7862595, false_negatives = 59.0, false_positives = 53.0, global_step = 643, loss = 0.47153282, precision = 0.7953668, recall = 0.7773585, true_negatives = 197.0, true_positives = 206.0


INFO:tensorflow:Saving dict for global step 643: auc = 0.7826792, eval_accuracy = 0.7825243, f1_score = 0.7862595, false_negatives = 59.0, false_positives = 53.0, global_step = 643, loss = 0.47153282, precision = 0.7953668, recall = 0.7773585, true_negatives = 197.0, true_positives = 206.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 643: /tmp/tmpl_v1i_zn/model.ckpt-643


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 643: /tmp/tmpl_v1i_zn/model.ckpt-643


{'auc': 0.7826792,
 'eval_accuracy': 0.7825243,
 'f1_score': 0.7862595,
 'false_negatives': 59.0,
 'false_positives': 53.0,
 'global_step': 643,
 'loss': 0.47153282,
 'precision': 0.7953668,
 'recall': 0.7773585,
 'true_negatives': 197.0,
 'true_positives': 206.0}

In [0]:
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.pandas_input_fn(
    train, train[LABEL_COLUMN], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.pandas_input_fn(
    test, test[LABEL_COLUMN], shuffle=False)

In [41]:
test_input_fn

<function bert.run_classifier.input_fn_builder.<locals>.input_fn>

In [0]:
def get_predictions(estimator, input_fn):
  return [x["label_ids"][0] for x in estimator.predict(input_fn=test_input_fn)]

In [44]:
LABELS = [
    "negative", "positive"
]

# Create a confusion matrix on training data.
with tf.Graph().as_default():
  cm = tf.confusion_matrix(train["mood_pn"], 
                           get_predictions(estimator, predict_train_input_fn))
  with tf.Session() as session:
    cm_out = session.run(cm)

# Normalize the confusion matrix so that each row sums to 1.
cm_out = cm_out.astype(float) / cm_out.sum(axis=1)[:, np.newaxis]

sns.heatmap(cm_out, annot=True, xticklabels=LABELS, yticklabels=LABELS);
plt.xlabel("Predicted");
plt.ylabel("True");

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


AssertionError: ignored